In [1]:
from my_trader import *
import math
from update_price import *

C:\ProgramData\Anaconda2\lib\site-packages\fix_yahoo_finance\__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and no longer available.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [3]:
robinhood = get_robinhood()
robinhood.login()

TypeError: login() takes exactly 3 arguments (1 given)

In [51]:
my_stock = pd.read_csv("file/Trade_suggestion" + str(datetime.now()-timedelta(days=1))[0:10]+".csv")

In [3]:
robinhood.get_my_position_beta_minute()

ZNGA 1.0000
TAHO 1.0000
RRGB 1.0000
SPY 6.0000
IEP 3.0000
OSG 1.0000
ROYT 2.0000
FCFS 1.0000
AES 1.0000
ISSC 1.0000
FDC 1.0000
WLK 1.0000
VRNT 1.0000
0.840984907799
          Beta  Quantity  Last_price    Weight
ZNGA  0.036493       1.0        3.80  0.001752
TAHO -0.369054       1.0        4.72  0.002176
RRGB  0.403548       1.0       53.30  0.024568
SPY   1.000000       6.0      279.61  0.773311
IEP   0.325061       3.0       55.90  0.077301
OSG   0.210234       1.0        2.81  0.001295
ROYT  0.105414       2.0        2.52  0.002323
FCFS  0.402574       1.0       70.95  0.032704
AES   0.222917       1.0       10.72  0.004941
ISSC  0.054599       1.0        3.03  0.001397
FDC   0.435015       1.0       17.60  0.008113
WLK   0.137103       1.0      108.77  0.050137
VRNT  0.405705       1.0       43.35  0.019982


(0.84098490779876256,           Beta  Quantity  Last_price    Weight
 ZNGA  0.036493       1.0        3.80  0.001752
 TAHO -0.369054       1.0        4.72  0.002176
 RRGB  0.403548       1.0       53.30  0.024568
 SPY   1.000000       6.0      279.61  0.773311
 IEP   0.325061       3.0       55.90  0.077301
 OSG   0.210234       1.0        2.81  0.001295
 ROYT  0.105414       2.0        2.52  0.002323
 FCFS  0.402574       1.0       70.95  0.032704
 AES   0.222917       1.0       10.72  0.004941
 ISSC  0.054599       1.0        3.03  0.001397
 FDC   0.435015       1.0       17.60  0.008113
 WLK   0.137103       1.0      108.77  0.050137
 VRNT  0.405705       1.0       43.35  0.019982)

In [15]:
beta(["UVXY"],bench= "^VIX",interval = "day")

(          Beta
 UVXY  0.846917, array([[ 0.00338583,  0.00206093],
        [ 0.00206093,  0.00174895]]))

In [60]:
robinhood.logout()

In [5]:
bench= "SPY"
ticker_list = ["UVXY"]
interval = "minute"

In [6]:
robinhood = get_robinhood()
betas = []
ben_mark=pd.DataFrame()
if interval == "day":
    ben_mark= da.DataReader(bench,"yahoo",datetime.now()-timedelta(days=90),datetime.now())
    ben_mark=ben_mark.rename(columns ={"Adj Close":bench})
elif interval == "minute":
    ben_mark = robinhood.get_historical(bench,interval="10minute",span = "week")
    ben_mark=ben_mark.rename(columns ={"close_price":bench})
    ben_mark[bench]=ben_mark[bench].astype(float)
    ben_mark.index = ben_mark.begins_at
ben_mark[bench + "_re"] = log(ben_mark[bench]/ben_mark[bench].shift(1))
for i in list(ticker_list):
    new=[]
    ticker = i
    try:
        if interval =="day":

            stock = da.DataReader(str(ticker),"yahoo",datetime.now()-timedelta(days=90),datetime.now())
            stock = stock.rename(columns = {"Adj Close":ticker})
        elif interval == "minute":
            stock= robinhood.get_historical(ticker,interval="10minute",span = "week")
            stock=stock.rename(columns ={"close_price":ticker})
            stock[ticker] = stock[ticker].astype(float)
            stock.index = stock.begins_at
    except:
        print (str(i)+" ticker maybe wrong. Error in getting data")
        betas.append(np.NaN)
        continue

    # get return and put them in a new dataframe

    stock[ticker + "_re"] = log(stock[ticker]/stock[ticker].shift(1))
    new = pd.concat([ben_mark,stock],axis =1)
    new = new[[bench + "_re",ticker + "_re" ]]
    new = new.dropna()

    #calculate beta using covariance matrix
    covmat = np.cov(new[bench + "_re"],new[ticker + "_re"])
    beta = covmat[0,1]/  np.sqrt(covmat[1,1]*covmat[0,0])
    betas.append(beta)
betas = pd.DataFrame(betas)
betas.index = ticker_list
betas.columns=["Beta"]


In [22]:
robinhood.my_trader.market_value()

2169.45

In [6]:
robinhood.my_trader.quotes_data(["AAPL","UVXY"])

[{u'adjusted_previous_close': u'176.1900',
  u'ask_price': u'179.4600',
  u'ask_size': 100,
  u'bid_price': u'179.3000',
  u'bid_size': 200,
  u'has_traded': True,
  u'instrument': u'https://api.robinhood.com/instruments/450dfc6d-5510-4d40-abfb-f633b7d9be3e/',
  u'last_extended_hours_trade_price': u'179.3700',
  u'last_trade_price': u'179.1000',
  u'last_trade_price_source': u'consolidated',
  u'previous_close': u'176.1900',
  u'previous_close_date': u'2018-01-16',
  u'symbol': u'AAPL',
  u'trading_halted': False,
  u'updated_at': u'2018-01-18T01:00:00Z'},
 {u'adjusted_previous_close': u'9.7500',
  u'ask_price': u'9.4600',
  u'ask_size': 1600,
  u'bid_price': u'9.4200',
  u'bid_size': 500,
  u'has_traded': True,
  u'instrument': u'https://api.robinhood.com/instruments/00e90099-4281-4c93-b50d-fbd4d2469821/',
  u'last_extended_hours_trade_price': u'9.4000',
  u'last_trade_price': u'9.3400',
  u'last_trade_price_source': u'consolidated',
  u'previous_close': u'9.7500',
  u'previous_close_

In [5]:
data = pd.read_csv("file/Trade_suggestion2018-01-17.csv")

In [6]:
data["position_place"]= np.ceil(((data.Close.sum()/len(data.Close))/data.Close))

In [44]:
np.dot(data.position_place,data.Close)

688.95499500000005

In [18]:
temp=robinhood.get_historical("AAPL",interval="10minute",span = "week")
temp.index = temp.begins_at
temp = temp.drop(["begins_at","session","interpolated"],axis=1)

In [32]:
tradeable = pd.read_csv("file/cantrade.csv")
tradeable = tradeable.dropna()
error = []
price=pd.DataFrame()

In [36]:
for i in list(tradeable.Ticker)[0:5]:
    trial = 0
    while trial <3:

        temp = robinhood.get_historical(i,interval="10minute",span = "week")
        temp.index = temp.begins_at
        temp = temp.drop(["begins_at","session","interpolated"],axis=1)
        for j in temp.columns:
            temp[j] = temp[j].astype(float)
        index= pd.MultiIndex.from_product([[i],temp.index])
        temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)
        price = price.append(temp)

        print "Finished", i 
        #time.sleep(5)
        trial=3

           

Finished AAPL
Finished BAC
Finished AMZN
Finished T
Finished GOOG


In [20]:
for i in temp.columns:
    temp[i] = temp[i].astype(float)

In [6]:
#price.to_csv("file/price_update.csv")
price = pd.read_csv("file/price_update.csv")

In [2]:
update_price_minute()


Finished AAPL
Finished BAC
Finished AMZN
Finished T
Finished GOOG
Finished MO
Finished DAL
Finished AA
Finished AXP
error occorded in getting yahool historicals for  DD
error occorded in getting yahool historicals for  DD
error occorded in getting yahool historicals for  DD
Finished BABA
Finished ABT
Finished UA
Finished AMAT
Finished AMGN
Finished AAL
Finished AIG
Finished ALL
Finished ADBE
Finished GOOGL
Finished ACN
Finished ABBV
Finished MT
Finished LLY
Finished AGN
Finished APA
Finished ADP
Finished APC
Finished AKAM
Finished NLY
Finished ABX
Finished ATVI
Finished ADSK
Finished ADM
Finished WBA
Finished ARNA
Finished LUV
Finished ACAD
Finished PANW
Finished AET
Finished AEP
Finished ALXN
Finished AVGO
Finished EA
Finished DB
error occorded in getting yahool historicals for  RAI
error occorded in getting yahool historicals for  RAI
error occorded in getting yahool historicals for  RAI
Finished AEM
Finished APD
Finished AMBA
Finished NVS
error occorded in getting yahool historicals

Finished ATR
Finished ASR
Finished ASC
Finished ARTNA
Finished AOD
Finished ADRO
Finished AAV
Finished AAT
Finished ZNH
error occorded in getting yahool historicals for  UAM
error occorded in getting yahool historicals for  UAM
error occorded in getting yahool historicals for  UAM
error occorded in getting yahool historicals for  NTT
error occorded in getting yahool historicals for  NTT
error occorded in getting yahool historicals for  NTT
Finished NFJ
Finished LNT
Finished KALU
Finished HOMB
Finished HIX
Finished FAF
Finished FAC
Finished BGR
Finished BETR
Finished ATV
Finished ATLC
Finished APLE
Finished AMSWA
Finished AMSF
Finished ALLT
Finished AGYS
Finished AFG
Finished WAIR
Finished LND
error occorded in getting yahool historicals for  ESD
error occorded in getting yahool historicals for  ESD
error occorded in getting yahool historicals for  ESD
Finished CII
error occorded in getting yahool historicals for  CHOP
error occorded in getting yahool historicals for  CHOP
error occorde

Finished BXS
Finished BOH
Finished BKU
Finished BKH
Finished BH
Finished BFAM
Finished BDN
Finished SBLK
Finished SBCF
Finished ROLL
Finished PSB
Finished PB
Finished HTBI
Finished GBDC
Finished FIZZ
Finished FFKT
Finished FCNCA
Finished BXE
Finished BWXT
Finished BMTC
Finished BLKB
Finished BBSI
Finished BABY
Finished WF
Finished OZRK
Finished JBT
Finished HDB
Finished FBP
Finished FBHS
error occorded in getting yahool historicals for  COB
error occorded in getting yahool historicals for  COB
error occorded in getting yahool historicals for  COB
Finished CBSH
Finished BWP
Finished BUSE
Finished BURL
Finished BRO
Finished BRKL
Finished BONT
Finished BOJA
Finished BGG
Finished BEAT
Finished BCC
Finished BANC
Finished WASH
Finished WABC
Finished UCBI
Finished PPBI
Finished NBTB
Finished INBK
Finished IBP
Finished EBR
Finished CSFL
Finished CSBK
Finished CIB
Finished CBU
Finished CATY
Finished BORN
Finished BOFI
Finished BLMN
Finished BBRG
Finished WIFI
Finished SJT
Finished SBS
Finished 

Finished CSII
Finished SINA
Finished FOXA
Finished CMI
Finished CCL
Finished MKC
Finished KHC
Finished NYT
Finished EW
Finished CI
Finished CAG
Finished WB
Finished VMC
Finished NWSA
error occorded in getting yahool historicals for  COH
error occorded in getting yahool historicals for  COH
error occorded in getting yahool historicals for  COH
Finished CLF
Finished NUE
Finished CHRW
Finished RL
Finished MCK
Finished LEN
Finished KKR
Finished HAIN
Finished DOV
Finished CTRP
Finished CNX
Finished CLDX
Finished CINF
Finished NCR
Finished HL
Finished NFX
Finished MNK
Finished LCI
Finished KMX
Finished HES
Finished TJX
Finished SE
Finished HUN
Finished HSY
Finished ETN
Finished NE
Finished L
Finished DHR
Finished CIM
error occorded in getting yahool historicals for  CSC
error occorded in getting yahool historicals for  CSC
error occorded in getting yahool historicals for  CSC
Finished CROX
Finished CHTR
Finished YGE
Finished WU
Finished PXD
Finished NAV
Finished EMN
error occorded in getting

Finished GRA
Finished GEOS
Finished FSIC
Finished FCN
Finished ECPG
Finished DLX
Finished CVEO
Finished COT
Finished CLI
Finished CHSP
Finished CHE
Finished CGI
Finished CCU
Finished CCK
Finished CCC
Finished CBT
Finished CALX
Finished CAFD
Finished CAC
Finished VICR
Finished UNT
Finished TEX
Finished SMG
Finished SEMG
Finished SATS
Finished RSYS
Finished RECN
Finished RCI
Finished POOL
Finished NEU
Finished MEOH
Finished MCC
Finished LOPE
Finished LNN
Finished LINC
Finished IBKC
Finished GPOR
Finished FSTR
error occorded in getting yahool historicals for  EVER
error occorded in getting yahool historicals for  EVER
error occorded in getting yahool historicals for  EVER
Finished DMRC
Finished CNS
Finished CLDT
Finished CLD
Finished CDK
error occorded in getting yahool historicals for  CBR
error occorded in getting yahool historicals for  CBR
error occorded in getting yahool historicals for  CBR
Finished CASH
Finished WLK
Finished VOXX
error occorded in getting yahool historicals for  VA

...

Finished NCA
Finished KAP
Finished HPF
error occorded in getting yahool historicals for  FPT
error occorded in getting yahool historicals for  FPT
error occorded in getting yahool historicals for  FPT
Finished FOF
Finished FMN
error occorded in getting yahool historicals for  FISH
error occorded in getting yahool historicals for  FISH
error occorded in getting yahool historicals for  FISH
Finished FHY
error occorded in getting yahool historicals for  FGL
error occorded in getting yahool historicals for  FGL
error occorded in getting yahool historicals for  FGL
Finished FFC
Finished FCB
Finished FAM
Finished WHF
Finished VGI
Finished TSLF
Finished PNF
Finished PHD
Finished NYV
Finished NUM
Finished NOMD
Finished NOM
Finished JPC
Finished JMM
Finished HPS
error occorded in getting yahool historicals for  GHI
error occorded in getting yahool historicals for  GHI
error occorded in getting yahool historicals for  GHI
Finished FMY
Finished FLC
Finished VGM
Finished THW
Finished SZC
Finished 

Finished JHI
Finished NSM
Finished MLHR
Finished KND
Finished ITUB
Finished HZO
Finished UVE
Finished RHP
Finished PRO
Finished PICO
Finished MHO
Finished HSTM
Finished HIBB
Finished TYPE
Finished MKTX
Finished JBHT
Finished HPP
error occorded in getting yahool historicals for  HNR
error occorded in getting yahool historicals for  HNR
error occorded in getting yahool historicals for  HNR
Finished TRQ
Finished SLGN
error occorded in getting yahool historicals for  ONE
error occorded in getting yahool historicals for  ONE
error occorded in getting yahool historicals for  ONE
Finished NOAH
Finished NMR
Finished NLSN
Finished INOV
Finished HNP
Finished HLIT
Finished HIL
Finished HI
Finished UHS
Finished TTS
error occorded in getting yahool historicals for  TMH
error occorded in getting yahool historicals for  TMH
error occorded in getting yahool historicals for  TMH
error occorded in getting yahool historicals for  SNOW
error occorded in getting yahool historicals for  SNOW
error occorded 

Finished WWW
Finished WTI
Finished WSBC
Finished WRI
Finished VEEV
Finished UFI
error occorded in getting yahool historicals for  TSYS
error occorded in getting yahool historicals for  TSYS
error occorded in getting yahool historicals for  TSYS
Finished TREE
Finished SWM
Finished STNG
Finished SPAR
Finished RTIX
Finished RMD
Finished PTC
Finished PRFT
Finished OMCL
Finished OIIM
Finished NXTM
Finished NVR
Finished NUVA
Finished NR
Finished LYTS
Finished LXRX
Finished LPSN
Finished KEYS
Finished IRWD
Finished IPI
Finished WMK
Finished WEX
Finished W
Finished TZOO
Finished SEAC
Finished SCHN
Finished RP
Finished ROP
Finished RNG
Finished PNM
Finished PGNX
Finished OSIS
Finished OMN
Finished MYE
Finished MTX
Finished MTEX
Finished MGA
error occorded in getting yahool historicals for  LDR
error occorded in getting yahool historicals for  LDR
error occorded in getting yahool historicals for  LDR
Finished KNX
Finished JOUT
Finished XBIT
Finished WING
Finished VEC
Finished SEDG
Finished QUAD


Finished PPR
Finished QEP
Finished QADA
Finished QUOT
Finished QADB
Finished VNO
Finished WSR
Finished SBR
error occorded in getting yahool historicals for  SALE
error occorded in getting yahool historicals for  SALE
error occorded in getting yahool historicals for  SALE
Finished RTEC
Finished TREC
Finished TCI
Finished RVT
Finished RUN
Finished RNWK
Finished RNET
Finished IRT
Finished RGNX
Finished SNY
Finished VALE
Finished SAP
Finished TEF
Finished TSE
Finished TS
Finished TX
Finished SCSC
Finished STAR
Finished SYX
Finished SFE
Finished STB
Finished SHOP
Finished ISSC
Finished VVR
Finished VRNS
Finished VCO
Finished SCM
Finished TWOU
Finished TRU
Finished TLGT
Finished TSRO
Finished VYGR
Finished VTL
Finished USAK
Finished TIME
Finished UTI
Finished TRVN
error occorded in getting yahool historicals for  TRIV
error occorded in getting yahool historicals for  TRIV
error occorded in getting yahool historicals for  TRIV
Finished TISI
Finished TTMI
Finished TLYS
Finished TESS
Finished C

AttributeError: 'DataFrame' object has no attribute 'High'

In [18]:
import pandas as pd
import numpy as np
import urllib2
import datetime as dt
import matplotlib.pyplot as plt
 
def get_google_data(symbol, period, window):
    url_root = 'http://www.google.com/finance/getprices?i='
    url_root += str(period) + '&amp;amp;p=' + str(window)
    url_root += 'd&amp;amp;f=d,o,h,l,c,v&amp;amp;df=cpct&amp;amp;q=' + symbol
    response = urllib2.urlopen(url_root)
    data = response.read().split('\n')
    #actual data starts at index = 7
    #first line contains full timestamp,
    #every other line is offset of period from timestamp
    parsed_data = []
    anchor_stamp = ''
    end = len(data)
    for i in range(7, end):
        cdata = data[i].split(',')
    if 'a' in cdata[0]:
    #first one record anchor timestamp
        anchor_stamp = cdata[0].replace('a', '')
        cts = int(anchor_stamp)
    else:
        try:
            coffset = int(cdata[0])
            cts = int(anchor_stamp) + (coffset * period)
            parsed_data.append((dt.datetime.fromtimestamp(float(cts)), float(cdata[1]), float(cdata[2]), float(cdata[3]), float(cdata[4]), float(cdata[5])))
        except:
            pass # for time zone offsets thrown into data
    df = pd.DataFrame(parsed_data)
    df.columns = ['ts', 'o', 'h', 'l', 'c', 'v']
    df.index = df.ts
    del df['ts']
    return df